# SmallNORB

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torchvision as tv
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from torchvision import transforms
from torchvision.transforms import v2
from datasets import load_dataset
from transformers import ConvNextV2ForImageClassification
from transformers.models.convnextv2.modeling_convnextv2 import ConvNextV2Embeddings

sns.set_theme()

c:\Users\daria\Desktop\multimodal-storytelling\.venv0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!nvidia-smi

Sat Nov 11 11:18:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.92       Driver Version: 528.92       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   39C    P8    13W /  50W |    662MiB /  8192MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
smallnorb = load_dataset("Ramos-Ramos/smallnorb")

In [ ]:
class SmallNORBDataset(torch.utils.data.Dataset):
  def __init__(self, hf_dataset, subset="train", transform=None):
    self.hf_dataset = hf_dataset
    self.subset = subset
    self.transform = transform

  def __len__(self):
    return len(self.hf_dataset[self.subset])

  def __getitem__(self, idx):
    sample = self.hf_dataset[self.subset][idx]
    image = sample["image_lt"]
    label = sample["category"]

    if self.transform:
      image = self.transform(image)

    return image, label

## VGG16

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size=(224,224), scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

transforms_val = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

transforms_test = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

In [ ]:
train_ds = SmallNORBDataset(smallnorb, "train", transform=transforms_train)
val_ds = SmallNORBDataset(smallnorb, "train", transform=transforms_val)

train_size = int(0.8 * len(train_ds))
val_size = len(train_ds) - train_size

train_ds, _ = torch.utils.data.random_split(train_ds, [train_size, val_size])
_, val_ds = torch.utils.data.random_split(val_ds, [train_size, val_size])

test_ds = SmallNORBDataset(smallnorb, "test", transform=transforms_test)

In [ ]:
IN_CHANNELS = 1
N_CLASSES = 5

vgg16_model = tv.models.vgg16(weights="IMAGENET1K_V1")

In [ ]:
for param in vgg16_model.parameters():
    param.requires_grad = False

In [ ]:
# 1-channel inputs
vgg16_model.features[0] = nn.Conv2d(IN_CHANNELS, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))

# Add on classifier
n_inputs = vgg16_model.classifier[6].in_features
vgg16_model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.6),
    nn.Linear(256, N_CLASSES), nn.LogSoftmax(dim=1))

In [ ]:
vgg16_model = vgg16_model.to(device)

In [ ]:
total_params = sum(p.numel() for p in vgg16_model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in vgg16_model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} trainable parameters.")

135,309,509 total parameters.
1,050,757 trainable parameters.


In [ ]:
BATCH_SIZE = 16
EPOCHS = 35
LEARNING_RATE = 1e-2
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4

In [ ]:
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = torch.utils.data.DataLoader(dataset=val_ds, batch_size=BATCH_SIZE)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=BATCH_SIZE)

In [ ]:
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(vgg16_model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode="max", factor=0.1, patience=11, verbose=True
)

In [ ]:
best_val_loss = 1e7
train_losses = list()
val_losses = list()

for epoch in range(EPOCHS):
  # Train
  vgg16_model.train()
  train_loss = 0.0
  for batch in tqdm(train_dl, desc=f"Epoch {epoch + 1}/{EPOCHS}", ncols=100):
    inputs, labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = vgg16_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  # Validate
  vgg16_model.eval()
  val_loss = 0.0
  correct_predictions = 0
  total_predictions = 0
  with torch.no_grad():
    for batch in tqdm(val_dl, desc="Validation", ncols=100):
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = vgg16_model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      total_predictions += labels.size(0)
      correct_predictions += (predicted == labels).sum().item()
  lr_scheduler.step(val_loss)

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(vgg16_model.state_dict(), "vgg16-smallnorb.pt")

  train_losses.append(train_loss / len(train_dl))
  val_losses.append(val_loss / len(val_dl))

  accuracy = 100 * correct_predictions / total_predictions
  print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss / len(train_dl):.4f}, Validation Loss: {val_loss / len(val_dl):.4f}, Validation Accuracy: {accuracy:.2f}%")


In [ ]:
# Predict on Test set
vgg16_model.eval()
test_loss = 0.0
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in tqdm(test_dl, desc="Testing", ncols=100):
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = vgg16_model(inputs)
      loss = criterion(outputs, labels)
      test_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      total_predictions += labels.size(0)
      correct_predictions += (predicted == labels).sum().item()

print(f"\nTest set accuracy = {(100 * correct_predictions / total_predictions):.4f}%")

Testing: 100%|██████████████████████████████████████████████████| 1519/1519 [01:37<00:00, 15.59it/s]


Test set accuracy = 91.2305%


## ConvNeXt

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]),
    transforms.RandomErasing(p=0.25)
])

transforms_val = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]),
])

transforms_test = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]),
])

In [ ]:
train_ds = SmallNORBDataset(smallnorb, "train", transform=transforms_train)
val_ds = SmallNORBDataset(smallnorb, "train", transform=transforms_val)

train_size = int(0.8 * len(train_ds))
val_size = len(train_ds) - train_size

train_ds, _ = torch.utils.data.random_split(train_ds, [train_size, val_size])
_, val_ds = torch.utils.data.random_split(val_ds, [train_size, val_size])

test_ds = SmallNORBDataset(smallnorb, "test", transform=transforms_test)

In [ ]:
IN_CHANNELS = 1
N_CLASSES = 5

convnext_model = tv.models.convnext_base(weights="IMAGENET1K_V1")

In [ ]:
for param in convnext_model.parameters():
    param.requires_grad = False

In [ ]:
# 1-channel inputs
convnext_model.features[0][0] = nn.Conv2d(IN_CHANNELS, 128, kernel_size=(4,4), stride=(4,4))

# Add on classifier
n_inputs = convnext_model.classifier[2].in_features
convnext_model.classifier[2] = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.GELU(), nn.Dropout(0.4),
    nn.Linear(256, N_CLASSES), nn.Softmax(dim=1))

In [ ]:
convnext_model = convnext_model.to(device)

In [ ]:
total_params = sum(p.numel() for p in convnext_model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in convnext_model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} trainable parameters.")

87,826,053 total parameters.
265,861 trainable parameters.


In [ ]:
BATCH_SIZE = 16
EPOCHS = 30
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 1e-8

In [ ]:
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = torch.utils.data.DataLoader(dataset=val_ds, batch_size=BATCH_SIZE)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=BATCH_SIZE)

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(convnext_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=11)

In [ ]:
# Additional ConvNeXt augmentations
cutmix = v2.CutMix(num_classes=N_CLASSES)
mixup = v2.MixUp(num_classes=N_CLASSES)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

In [ ]:
best_val_loss = 1e7
train_losses = list()
val_losses = list()

for epoch in range(EPOCHS):
  # Train
  convnext_model.train()
  train_loss = 0.0
  for batch in tqdm(train_dl, desc=f"Epoch {epoch + 1}/{EPOCHS}", ncols=100):
    inputs, labels = batch
    inputs, labels = cutmix_or_mixup(inputs, labels)
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = convnext_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  # Validate
  convnext_model.eval()
  val_loss = 0.0
  correct_predictions = 0
  total_predictions = 0
  with torch.no_grad():
    for batch in tqdm(val_dl, desc="Validation", ncols=100):
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = convnext_model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      total_predictions += labels.size(0)
      correct_predictions += (predicted == labels).sum().item()
  lr_scheduler.step()

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(convnext_model.state_dict(), "convnext-smallnorb.pt")

  train_losses.append(train_loss / len(train_dl))
  val_losses.append(val_loss / len(val_dl))

  accuracy = 100 * correct_predictions / total_predictions
  print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss / len(train_dl):.4f}, Validation Loss: {val_loss / len(val_dl):.4f}, Validation Accuracy: {accuracy:.2f}%")


In [ ]:
# Predict on Test set
convnext_model.eval()
test_loss = 0.0
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in tqdm(test_dl, desc="Testing", ncols=100):
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = convnext_model(inputs)
      loss = criterion(outputs, labels)
      test_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      total_predictions += labels.size(0)
      correct_predictions += (predicted == labels).sum().item()

print(f"\nTest set accuracy = {(100 * correct_predictions / total_predictions):.4f}%")

Testing: 100%|██████████████████████████████████████████████████| 1519/1519 [02:43<00:00,  9.28it/s]


Test set accuracy = 92.4979%


## ConvNeXt V2

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]),
    transforms.RandomErasing(p=0.25)
])

transforms_val = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]),
])

transforms_test = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229]),
])

In [ ]:
train_ds = SmallNORBDataset(smallnorb, "train", transform=transforms_train)
val_ds = SmallNORBDataset(smallnorb, "train", transform=transforms_val)

train_size = int(0.8 * len(train_ds))
val_size = len(train_ds) - train_size

train_ds, _ = torch.utils.data.random_split(train_ds, [train_size, val_size])
_, val_ds = torch.utils.data.random_split(val_ds, [train_size, val_size])

test_ds = SmallNORBDataset(smallnorb, "test", transform=transforms_test)

In [ ]:
IN_CHANNELS = 1
N_CLASSES = 5

convnextv2_model = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-base-1k-224")

In [ ]:
for param in convnextv2_model.parameters():
    param.requires_grad = False

In [ ]:
# 1-channel inputs
convnextv2_model.convnextv2.embeddings.patch_embeddings = nn.Conv2d(IN_CHANNELS, 128, kernel_size=(4,4), stride=(4,4))
convnextv2_model.convnextv2.embeddings.num_channels = IN_CHANNELS

# Add on classifier
n_inputs = convnextv2_model.classifier.in_features
convnextv2_model.classifier = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.GELU(), nn.Dropout(0.4),
    nn.Linear(256, N_CLASSES), nn.Softmax(dim=1))

In [ ]:
convnextv2_model = convnextv2_model.to(device)

In [ ]:
total_params = sum(p.numel() for p in convnextv2_model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in convnextv2_model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} trainable parameters.")

87,952,389 total parameters.
265,861 trainable parameters.


In [ ]:
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 6.25e-3
WEIGHT_DECAY = 0.05

In [ ]:
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = torch.utils.data.DataLoader(dataset=val_ds, batch_size=BATCH_SIZE)
test_dl = torch.utils.data.DataLoader(dataset=test_ds, batch_size=BATCH_SIZE)

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(convnextv2_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=11)

In [ ]:
# Additional ConvNeXt augmentations
cutmix = v2.CutMix(num_classes=N_CLASSES)
mixup = v2.MixUp(alpha=0.8, num_classes=N_CLASSES)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

In [ ]:
best_val_loss = 1e7
train_losses = list()
val_losses = list()

for epoch in range(EPOCHS):
  # Train
  convnextv2_model.train()
  train_loss = 0.0
  for batch in tqdm(train_dl, desc=f"Epoch {epoch + 1}/{EPOCHS}", ncols=100):
    inputs, labels = batch
    inputs, labels = cutmix_or_mixup(inputs, labels)
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = convnextv2_model(inputs)
    loss = criterion(outputs[0], labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  # Validate
  convnextv2_model.eval()
  val_loss = 0.0
  correct_predictions = 0
  total_predictions = 0
  with torch.no_grad():
    for batch in tqdm(val_dl, desc="Validation", ncols=100):
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = convnextv2_model(inputs)
      loss = criterion(outputs[0], labels)
      val_loss += loss.item()
      _, predicted = torch.max(outputs[0], 1)
      total_predictions += labels.size(0)
      correct_predictions += (predicted == labels).sum().item()
  lr_scheduler.step()

  if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(convnextv2_model.state_dict(), "convnextv2-smallnorb.pt")

  train_losses.append(train_loss / len(train_dl))
  val_losses.append(val_loss / len(val_dl))

  accuracy = 100 * correct_predictions / total_predictions
  print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss / len(train_dl):.4f}, Validation Loss: {val_loss / len(val_dl):.4f}, Validation Accuracy: {accuracy:.2f}%")


In [ ]:
# Predict on Test set
convnextv2_model.eval()
test_loss = 0.0
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in tqdm(test_dl, desc="Testing", ncols=100):
      inputs, labels = batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = convnextv2_model(inputs)
      loss = criterion(outputs[0], labels)
      test_loss += loss.item()
      _, predicted = torch.max(outputs[0], 1)
      total_predictions += labels.size(0)
      correct_predictions += (predicted == labels).sum().item()

print(f"\nTest set accuracy = {(100 * correct_predictions / total_predictions):.4f}%")

Testing: 100%|██████████████████████████████████████████████████| 1519/1519 [03:43<00:00,  6.80it/s]


Test set accuracy = 92.5267%


## Improved ResNet50

## Swin Transformer